In [24]:
import copy
import open3d as o3d
import numpy as np
#====================读取点云数据===================
source = o3d.io.read_point_cloud("./mapping/sample_0.ply")
target = o3d.io.read_point_cloud("./mapping/sample_1.ply")


In [25]:
#==================可视化点云初始位置===============
o3d.visualization.draw_geometries([source, target],width=600,height=600)
threshold = 0.2 #距离阈值
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0],
                         [0.0, 0.0, 0.0, 1.0]]) #初始变换矩阵，一般由粗配准提供

In [26]:
#=================================================
#计算两个重要指标，fitness计算重叠区域（内点对应关系/目标点数）。越高越好。
#inlier_rmse计算所有内在对应关系的均方根误差RMSE。越低越好。
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)#这里输出的是初始位置的 fitness和RMSE
print("Apply point-to-point ICP")
icp_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),#执行点对点的ICP算法
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=50))#设置最大迭代次数
print(icp_p2p)#输出ICP相关信息
print("Transformation is:")
print(icp_p2p.transformation)#输出变换矩阵

Initial alignment
RegistrationResult with fitness=2.478389e-01, inlier_rmse=1.172079e-01, and correspondence_set size of 74427
Access transformation to get result.
Apply point-to-point ICP
RegistrationResult with fitness=4.073872e-01, inlier_rmse=1.040227e-01, and correspondence_set size of 122340
Access transformation to get result.
Transformation is:
[[ 0.95103678 -0.16441565 -0.26171842 -0.89163941]
 [ 0.24706233  0.91319294  0.32409698  0.24499192]
 [ 0.18571279 -0.37288891  0.90910099 -0.1835678 ]
 [ 0.          0.          0.          1.        ]]


In [27]:
#================可视化配准结果====================
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)       #由于函数transformand paint_uniform_color会更改点云，
    target_temp = copy.deepcopy(target)       #因此调用copy.deepcoy进行复制并保护原始点云。
#     source_temp.paint_uniform_color([1, 0, 0])#点云着色
#     target_temp.paint_uniform_color([0, 1, 0])
    source_temp.transform(transformation)
#     o3d.io.write_point_cloud("out.ply", source_temp)#保存点云
    o3d.visualization.draw_geometries([source_temp, target_temp],width=600,height=600)
draw_registration_result(source, target, icp_p2p.transformation)

## METHOD 2 ##


In [3]:
o3d.visualization.draw_geometries([source])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [8]:
threshold = 1.0  #移动范围的阀值
trans_init = np.asarray([[1,0,0,0],   # 4x4 identity matrix，这是一个转换矩阵，
                         [0,1,0,0],   # 象征着没有任何位移，没有任何旋转，我们输入
                         [0,0,1,0],   # 这个矩阵为初始变换
                         [0,0,0,1]])

In [12]:
#运行icp
reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

In [14]:
print(reg_p2p)
source.transform(reg_p2p.transformation)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.578777e-01, and correspondence_set size of 300304
Access transformation to get result.


PointCloud with 300304 points.

In [15]:
o3d.visualization.draw_geometries([source,target])

## METHOD 3 ##


In [12]:
#====================读取点云数据===================
source = o3d.io.read_point_cloud("./pcd/s_300.ply")
target = o3d.io.read_point_cloud("./pcd/s_350.ply")

In [13]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [14]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [15]:
def prepare_dataset(source,target, voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
#     source = o3d.io.read_point_cloud("../../TestData/ICP/cloud_bin_0.pcd")
#     target = o3d.io.read_point_cloud("../../TestData/ICP/cloud_bin_1.pcd")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

voxel_size = 0.05 # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source,target,voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


我们使用RANSAC进行全局配准.在RANSAC迭代中，我们每次从源点云中选取　ransac_n 个随机点.通过在33维FPFH特征空间中查询最邻近,可以在目标点云中找到他们的对应点.剪枝步骤需要使用快速修剪算法来提早拒绝错误匹配.
Open3d提供以下剪枝算法:

CorrespondenceCheckerBasedOnDistance检查对应的点云是否接近(也就是距离是否小于指定阈值)

CorrespondenceCheckerBasedOnEdgeLength检查从源点云和目标点云对应中分别画上两条任意边(两个顶点连成的线)是否近似.

CorrespondenceCheckerBasedOnNormal考虑的所有的对应的顶点法线的密切关系.他计算了两个法线向量的点积.使用弧度作为阈值.

只有通过剪枝步骤的匹配才用于转换,该转换将在整个点云上进行验证.核心函数是 ：

registration_ransac_based_on_feature_matching. 

RANSACConvergenceCriteria是里面一个十分重要的超参数.他定义了RANSAC迭代的最大次数和验证的最大次数.这两个值越大,那么结果越准确,但同时也要花费更多的时间.
我们是基于[Choi2015]提供的的经验来设置RANSAC的超参数.

In [16]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: 对下采样的点云进行RANSAC配准.")
    print("   下采样体素的大小为： %.3f," % voxel_size)
    print("   使用宽松的距离阈值： %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
         ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result




In [17]:
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: 对下采样的点云进行RANSAC配准.
   下采样体素的大小为： 0.050,
   使用宽松的距离阈值： 0.075.
RegistrationResult with fitness=2.654012e-01, inlier_rmse=4.487812e-02, and correspondence_set size of 1538
Access transformation to get result.


In [23]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: 对原始点云进行点对面ICP配准精细对齐， 这次使用严格的距离阈值： %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(source, target, distance_threshold,                                            result_ransac.transformation,o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result


result_icp = refine_registration(source, target, source_fpfh, target_fpfh, voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: 对原始点云进行点对面ICP配准精细对齐， 这次使用严格的距离阈值： 0.020.


RuntimeError: [Open3D Error] (class open3d::pipelines::registration::RegistrationResult __cdecl open3d::pipelines::registration::RegistrationICP(const class open3d::geometry::PointCloud &,const class open3d::geometry::PointCloud &,double,const class Eigen::Matrix<double,4,4,0,4,4> &,const class open3d::pipelines::registration::TransformationEstimation &,const class open3d::pipelines::registration::ICPConvergenceCriteria &)) D:\a\Open3D\Open3D\cpp\open3d\pipelines\registration\Registration.cpp:147: TransformationEstimationPointToPlane and TransformationEstimationColoredICP require pre-computed normal vectors for target PointCloud.


## METHOD 4 ##

In [27]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
#     source_temp.paint_uniform_color([1, 0.706, 0])
#     target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [36]:
o3d.visualization.draw_geometries([source, target])

In [28]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: 使用大小为为{}的体素下采样点云.".format(voxel_size))
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: 使用搜索半径为{}估计法线".format(radius_normal))
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: 使用搜索半径为{}计算FPFH特征".format(radius_feature))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [29]:
def prepare_dataset(voxel_size):
    print(":: 加载点云并转换点云的位姿.")
    source = o3d.io.read_point_cloud("./pcd/s_300.ply")
    target = o3d.io.read_point_cloud("./pcd/s_325.ply")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

# 相当于使用5cm的体素对点云进行均值操作
voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size)

:: 加载点云并转换点云的位姿.
:: 使用大小为为0.05的体素下采样点云.
:: 使用搜索半径为0.1估计法线
:: 使用搜索半径为0.25计算FPFH特征
:: 使用大小为为0.05的体素下采样点云.
:: 使用搜索半径为0.1估计法线
:: 使用搜索半径为0.25计算FPFH特征


In [30]:
def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: 对下采样的点云进行RANSAC配准.")
    print("   下采样体素的大小为： %.3f," % voxel_size)
    print("   使用宽松的距离阈值： %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True, distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 3,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
         ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result


result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: 对下采样的点云进行RANSAC配准.
   下采样体素的大小为： 0.050,
   使用宽松的距离阈值： 0.075.
RegistrationResult with fitness=5.680759e-01, inlier_rmse=3.266798e-02, and correspondence_set size of 3292
Access transformation to get result.


In [35]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: 对原始点云进行点对面ICP配准精细对齐， 这次使用严格的距离阈值： %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(source, target, distance_threshold,
                                                         result_ransac.transformation,
#                     target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.08, max_nn=30)),
                                                    o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result


result_icp = refine_registration(source, target, source_fpfh, target_fpfh, voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: 对原始点云进行点对面ICP配准精细对齐， 这次使用严格的距离阈值： 0.020.
RegistrationResult with fitness=3.318262e-01, inlier_rmse=6.013024e-03, and correspondence_set size of 101937
Access transformation to get result.
